# Data Transformation
The purpose of this notebook is to transform the raw waze.csv data into a format matching our data schema. Before running this notebook, please run the following SQL script to create a new database:

CREATE DATABASE waze_schema
    WITH 
    OWNER = postgres
    ENCODING = 'UTF8'
    CONNECTION LIMIT = -1;

CREATE EXTENSION postgis;
CREATE EXTENSION postgis_topology;

## Load Raw Data

In [ ]:
import datetime
import numpy as np
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import shapely
import time
from sqlalchemy import create_engine
import ast

### set parameters 

In [ ]:
start_time = time.time()

In [ ]:
args_file = '../conf/pipeline_args.txt'
fr = open(args_file, 'r')
fa = fr.read()
file_args = ast.literal_eval(fa)
file_args

In [ ]:
sqlalchemy_conn_str = open('../conf/sqlalchemy_conn_str.txt', 'r').read()
engine = create_engine(sqlalchemy_conn_str)

In [ ]:
postgres_password = open('../conf/postgres_password.txt', 'r').read()
conn_str = "host={} dbname={} user={} password={}".format(
    'localhost', 'waze_schema', 'postgres', (*your db username here*))

conn = pg.connect(conn_str)
cur = conn.cursor()

In [ ]:
time_bucket = str(file_args['time_resolution'])
filepath = '../data/'
cum_ts = 100/file_args['time_queries']['cum_ts_pct']
cum_seg = 100/file_args['segment_queries']['cum_seg_pct']

In [ ]:
# allow processing the raw fcsv ile directly or importing the csv into the db first 
# then re-pulling the data for transformation.
import_type = 'csv' # db or csv

if import_type == 'csv':
    csv_file = filepath + 'waze_data.csv'
    waze_raw_df = pd.read_csv(csv_file)
elif import_type == 'db':  # assumes postgresql
    # assume connection file is always present
    conn_str_file = 'db_conn_str.txt'
    conn = pg.connect(pg_conn_str)
    waze_raw_df = pd.read_sql('select  id, uuid, waze_timestamp, street, \
                                       start_node, end_node, city, length, delay, \
                                       speed, level, road_type, geom, \
                                       ST_AsText(geom) as linestring, \
                                       ST_NumPoints(geom) as linestring_length \
                               from congestion', con=conn)

# make a copy of waze_raw_df
waze_processed_df = waze_raw_df.copy()

In [ ]:
waze_processed_df.insert(0, 'id', range(1, len(waze_processed_df)+1))

In [ ]:
#waze_processed_df = waze_processed_df[waze_processed_df['uuid']<=1000000].reset_index().drop(columns = ['index'])

## Create Processed DataFrame with Additional Columns

In [ ]:
waze_processed_df.head()

In [ ]:
# extract day of week, date, time, and timestamp rounded to 15 minute interval
if import_type == 'csv':
    waze_processed_df['dow'] = pd.to_datetime(waze_processed_df['waze_timestamp']).dt.dayofweek
    waze_processed_df['month'] = pd.to_datetime(waze_processed_df['waze_timestamp']).dt.month
    waze_processed_df['date'] = pd.to_datetime(waze_processed_df['waze_timestamp']).dt.date
    waze_processed_df['time'] = pd.to_datetime(waze_processed_df['waze_timestamp']).dt.time
    waze_processed_df['timestamp_round'] = pd.to_datetime(waze_processed_df['waze_timestamp']).apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,5*(dt.minute // 5)))
    waze_processed_df['time_round'] = pd.to_datetime(waze_processed_df['timestamp_round']).dt.time
elif import_type == 'db':
    waze_processed_df['dow'] = waze_processed_df['waze_timestamp'].dt.dayofweek
    waze_processed_df['month'] = waze_processed_df['waze_timestamp'].dt.month
    waze_processed_df['date'] = waze_processed_df['waze_timestamp'].dt.date
    waze_processed_df['time'] = waze_processed_df['waze_timestamp'].dt.time
    waze_processed_df['timestamp_round'] = waze_processed_df['waze_timestamp'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,5*(dt.minute // 5)))
    waze_processed_df['time_round'] = waze_processed_df['timestamp_round'].dt.time 

In [ ]:
waze_processed_df['is_weekend'] = np.where(((waze_processed_df['dow']==5) | (waze_processed_df['dow']==6)), 1, None)

holidays = [datetime.date(2017,1,2), datetime.date(2017,1,16), datetime.date(2017,2,20), datetime.date(2017,5,29)]
holidays_df = pd.DataFrame(holidays, columns = ['date'])
holidays_df['is_holiday'] = 1
waze_processed_df = pd.merge(waze_processed_df, holidays_df, how = 'left', on= 'date')

waze_processed_df['waze_timestamp_tmp'] = pd.to_datetime(waze_processed_df['waze_timestamp'])
waze_processed_df['is_rushhour'] = np.where((waze_processed_df.waze_timestamp_tmp.dt.strftime('%H:%M:%S').between('07:00:00','09:00:00')) | (waze_processed_df.waze_timestamp_tmp.dt.strftime('%H:%M:%S').between('16:00:00','19:00:00')), 1, None)
waze_processed_df = waze_processed_df.drop(columns = ['waze_timestamp_tmp'])

In [ ]:
waze_processed_df.shape

In [ ]:
from shapely.geometry import LineString, shape
from shapely.wkb import dumps, loads
from shapely.wkt import dumps, loads

In [ ]:
def linestring_length(row):
    _,ls = row.split('(')
    linestring = ls[:-1]
    segments = linestring.split(',')
    return len(segments)

In [ ]:
if import_type == 'csv':
    waze_processed_df['linestring'] = \
        waze_processed_df['geom'].apply(lambda x: shapely.wkb.loads(x, hex=True).wkt)

    waze_processed_df['linestring_length'] =  waze_processed_df['linestring'].apply(linestring_length)
waze_processed_df.head()

In [ ]:
waze_processed_df.shape

In [ ]:
print('--- reading and processing csv results took {0:.1f} seconds ---'.format(time.time() - start_time))

## Create Time Table

In [ ]:
time_cols = ['timestamp_round','date','time_round','dow','month','is_weekend','is_holiday','is_rushhour']
time_df = waze_processed_df.loc[:,time_cols].drop_duplicates().sort_values(by='timestamp_round').reset_index()
time_df.drop('index', axis=1, inplace=True)
time_df['time_id'] = time_df.index + 1
time_df.columns = ['timestamp_round', 'date', 'time', 'day_of_week', 'month', 'is_weekend', 'is_holiday', 'is_rushhour', 'time_id']
time_df = time_df[['date','day_of_week','month','is_weekend','is_holiday','is_rushhour','time','timestamp_round','time_id']]
time_df.head()

## Create Segment Table

In [ ]:
# helper function
def extract_segments(ls):
    '''function to extract individual lonlat segments from a linestring'''
    ls_0 = ls.split('(')[-1:][0]
    lonlats_str = ls_0[:-1].split(',')
    lons = [float(ll.split()[0]) for ll in lonlats_str]
    lats = [float(ll.split()[1]) for ll in lonlats_str]
    lonlats = list(zip(lons, lats))
    segments = [(lonlats[i],lonlats[i+1]) for i in range(len(lonlats)-1)]
    return segments

In [ ]:
# extract segments from linestrings
linestrings = np.array(waze_processed_df['linestring'].values)
waze_segments = map(extract_segments, linestrings)

In [ ]:
# check to make sure lengths align
# segment_lengths = np.array(map(len, waze_segments)) + 1
# linestring_lengths = np.array(waze_raw_df['linestring_length'].values)

# print 'should be zero: {}'.format(np.average(linestring_lengths - segment_lengths))

In [ ]:
# add segments to processed dataframe
waze_processed_df['segments'] = waze_segments

In [ ]:
# # check random rows to make sure linestrings and segments match
# nonmatching_row_count = 0
# for random_row in np.random.choice(waze_processed_df.index.values, size=1000, replace=False):
#     rand_linestring = waze_processed_df.iloc[random_row]['linestring']
#     rand_segments = waze_processed_df.iloc[random_row]['segments']
#     if rand_segments == extract_segments(rand_linestring):
#         continue
#     else:
#         nonmatching_row_count+=1
#         print 'row {} does not match'.format(random_row)

# print 'nonmatching rows: {}'.format(nonmatching_row_count)

In [ ]:
# get list of individual segments
segments_list = waze_processed_df['segments'].values

In [ ]:
# flatten list
flat_segments_list = [segment for segments in segments_list for segment in segments]

In [ ]:
# get unique segments
unique_segments = np.array(list(set(flat_segments_list)))

In [ ]:
# create dict for dataframe
segments_dict = {
    'segment_id': np.array(range(len(unique_segments))) + 1,
    'segment': [s for s in unique_segments],
    'lat1': [s[0][1] for s in unique_segments],
    'lon1': [s[0][0] for s in unique_segments],
    'lat2': [s[1][1] for s in unique_segments],
    'lon2': [s[1][0] for s in unique_segments]
}

In [ ]:
# create segment dataframe
segment_df = pd.DataFrame(segments_dict)

In [ ]:
# # TODO - add additional columns
# segment_df['street'] = None
# segment_df['city'] = None
# segment_df['road_type'] = None
# segment_df['geom'] = None
# segment_df['length'] = None

In [ ]:
segment_df_tmp = segment_df[['lat1','lon1','lat2','lon2','segment_id']]

## Create UUID Table

In [ ]:
waze_processed_df['uuid_instance_id'] = waze_processed_df.index
uuid_df = waze_processed_df[['uuid', 'uuid_instance_id', 'street','start_node','end_node','waze_timestamp','city','length','delay','speed','level','road_type']]
uuid_df.columns = ['uuid', 'uuid_instance_id', 'street_original', 'start_node', 'end_node', 'waze_timestamp', 'city_original', 'length_original', 'delay', 'speed', 'level', 'road_type_original']
uuid_df.head()

## Create Time/Segment/UUID Mapping Table

In [ ]:
matrix_df = pd.DataFrame(columns=['uuid','uuid_instance_id','segments','lat1','lon1','lat2','lon2','path'])

for i in range(len(waze_processed_df)):
    linestring = waze_processed_df['segments'][i] 
    tmp_df = pd.DataFrame(linestring) 
    tmp_df['uuid'] = waze_processed_df['uuid'][i] 
    tmp_df['uuid_instance_id'] = waze_processed_df['uuid_instance_id'][i]
    tmp_df['timestamp_round'] = waze_processed_df['timestamp_round'][i]
    tmp_df['segments'] = waze_processed_df['segments'][i]

    lon1 = []
    lat1 = []
    lon2 = []
    lat2 = []
    for j in range(len(tmp_df)):
        lon1.append(tmp_df[0][j][0])
        lat1.append(tmp_df[0][j][1])
        lon2.append(tmp_df[1][j][0])
        lat2.append(tmp_df[1][j][1])
    tmp_df['lon1'] = lon1
    tmp_df['lat1'] = lat1
    tmp_df['lon2'] = lon2
    tmp_df['lat2'] = lat2

    tmp_df.insert(0, 'path', range(len(tmp_df)))
    tmp_df
    matrix_df = matrix_df.append(tmp_df)

In [ ]:
matrix_df_tmp = pd.merge(matrix_df, time_df, on='timestamp_round')
matrix_df_tmp2 = pd.merge(matrix_df_tmp, segment_df, on=['lat1','lon1','lat2','lon2'])
matrix_df = matrix_df_tmp2[['uuid_instance_id', 'path', 'time_id', 'segment_id']]
matrix_df.head()

## Update Segments table

In [ ]:
uuid_withseg = pd.merge(uuid_df, matrix_df[['segment_id','uuid_instance_id']], on=['uuid_instance_id'])
segfields = uuid_withseg[['segment_id','street_original','city_original','road_type_original']].drop_duplicates()
segments_df = pd.merge(segment_df_tmp, segfields, on='segment_id')
segments_df.columns = ['lat1','lon1','lat2','lon2','segment_id','street','city','road_type']
segments_df.head()

## Write DataFrames to Postgres

In [ ]:
time_df.to_sql(name='time', con=engine, if_exists='replace', dtype={'is_rushhour': sqlalchemy.types.Boolean, 
                             'is_weekend':  sqlalchemy.types.Boolean,
                             'is_holiday': sqlalchemy.types.Boolean,
                             'day_of_week': sqlalchemy.types.String})
segments_df.to_sql(name='segments', con=engine, if_exists='replace')
uuid_df.to_sql(name='uuid', con=engine, if_exists='replace')
matrix_df.to_sql(name='matrix', con=engine, if_exists='replace')

In [ ]:
print('--- reading, processing csv and inserting results into db took {0:.1f} seconds ---'.format(time.time() - start_time))

## Write Padres Games DataFrame to Postgres

In [ ]:
padres_df = pd.read_csv(filepath+'padreswindow.csv')
padres_df = padres_df[['Date','start_time','Time','Attendance']]

In [ ]:
def fix_date(d):
    mon, day = d.split()[1:]
    if len(day) == 1:
        day = '0'+day
    return '{} {} 2017'.format(mon, day)

In [ ]:
def create_timedelta(duration):
    hrs_mins = [int(x) for x in duration.split(':')]
    mins = 60*hrs_mins[0] + hrs_mins[1]
    return pd.Timedelta(minutes = mins)

In [ ]:
fixed_dates = padres_df['Date'].apply(fix_date)
padres_df.loc[:,'Date'] = fixed_dates

date_col = pd.to_datetime(padres_df['Date'])
padres_df.loc[:,'Date'] = date_col

start_time_col = pd.to_datetime(padres_df['start_time'], format='%H:%M').dt.time
padres_df.loc[:,'start_time'] = start_time_col

game_duration = padres_df['Time'].apply(create_timedelta)
padres_df.loc[:,'game_duration'] = game_duration

padres_df['game_start'] = padres_df[['Date','start_time']].apply(lambda row: datetime.datetime.combine(row['Date'], row['start_time']), axis=1)
padres_df['game_end'] = padres_df['game_start'] + padres_df['game_duration']

padres_df = padres_df[['game_start','game_end','Attendance']]
padres_df.columns = ['game_start','game_end','attendance']

padres_df.head()

In [ ]:
padres_df.to_sql(name='padres_games', con=engine, if_exists='replace', index=False)

## SQL Updates

In [ ]:
exec(open("SQLQueries.py").read())

In [ ]:
cur.execute(SQL_drop_indexes)

In [ ]:
cur.execute(SQL_update_segments)

In [ ]:
cur.execute(SQL_create_events_table)

In [ ]:
cur.execute(SQL_Time_Bucketing)

In [ ]:
cur.execute(SQL_create_segments_times_selected)

In [ ]:
cur.execute(SQL_pct_segments)

In [ ]:
cur.execute(SQL_pct_time)

In [ ]:
conn.commit()
cur.close()
conn.close()